In [44]:
import os
import sys



os.chdir('/Users/yashdive/ML Repo/Portfolio Projects/poultry-disease-classifier/poultry-disease-classifier/research'
)
os.getcwd()
sys.path.append(os.path.abspath(os.path.join("..", "src")))

os.chdir("../")
%pwd
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

from henalyze_CNN_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from henalyze_CNN_classifier.utils.common import read_yaml, create_directories


In [45]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [47]:
import os
import urllib.request as request
import zipfile
from henalyze_CNN_classifier import logger
from henalyze_CNN_classifier.utils.common import get_size

In [48]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [50]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-16 12:53:10,670 - INFO - common - yaml file: config/config.yaml loaded successfully]
[2025-07-16 12:53:10,672 - INFO - common - yaml file: params.yaml loaded successfully]
[2025-07-16 12:53:10,673 - INFO - common - created directory at: artifacts]
[2025-07-16 12:53:10,673 - INFO - common - created directory at: artifacts/data_ingestion]
[2025-07-16 12:53:20,677 - INFO - 4037744573 - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 08D7:9CE80:CF032:1F5C06:6877534A
Accept-Ranges: bytes
Date: Wed, 16 Jul 2025 07:23:11 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4734-B